In [1]:


%pip install ffmpeg-python
%pip install transformers

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached future-0.18.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
Using cached transformers-4.32.0-py3-none-any.whl (7.5 MB)
Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
Note: you may need to restart the kernel to use updated packages.


## Imports

## 0. Record or Import Audio

In [ ]:
# @title Run audio recording script
## script created by Eric Lam
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

import io
import wave
from base64 import b64decode

import ffmpeg
import numpy as np
from google.colab.output import eval_js
from IPython.display import HTML
from scipy.io.wavfile import read as wav_read


def write_wav(f, sr, x, normalized=False):
    f = wave.open(f, "wb")
    f.setnchannels(1)
    f.setsampwidth(2)
    f.setframerate(sr)

    wave_data = x.astype(np.short)
    f.writeframes(wave_data.tobytes())
    f.close()


def get_audio():

    # call microphone
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(",")[1])

    process = (
        ffmpeg.input("pipe:0")
        .output("pipe:1", format="wav")
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, _ = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]
    sr, audio = wav_read(io.BytesIO(riff))
    # save
    human_sound_file = "demo.wav"
    write_wav(human_sound_file, sr, audio)

get_audio()

In [ ]:
import wave
import numpy as np

def get_record(path: str):
  # Open the WAV file
  wav_file_path = path
  wav_file = wave.open(wav_file_path, 'rb')
  # Get the parameters of the WAV file
  num_channels = wav_file.getnchannels()
  sample_width = wav_file.getsampwidth()
  frame_rate = wav_file.getframerate()
  num_frames = wav_file.getnframes()
  # Read the audio data from the WAV file
  raw_data = wav_file.readframes(num_frames)
  # Convert the raw data to a numpy array of integers
  int_data = np.frombuffer(raw_data, dtype=np.int16)
  # Normalize the integer data to the range of [-1, 1]
  float_data = int_data / (2 ** 15)
  # Close the WAV file
  wav_file.close()

  return float_data


## 1. Speech recoginition (Arabic --> Arabic)
Using OpenAI/WhisperLargeV2

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("bardsai/whisper-medium-pl-v2")
model = AutoModelForSpeechSeq2Seq.from_pretrained("bardsai/whisper-medium-pl-v2")

inputs = processor(get_record("demo.wav"),return_tensors="pt")

In [ ]:
from transformers import AutoModelForCTC
import torch

model = AutoModelForCTC.from_pretrained("stevhliu/my_awesome_asr_mind_model")
with torch.no_grad():
    logits = model(**inputs).logits